In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import regex as re

In [41]:
# Extracting Movie Links

link = "https://www.imdb.com/chart/top/"
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
response = requests.get(link, headers=agent)
soup = BeautifulSoup(response.content, 'html.parser')
#print(soup.prettify())
movie_container = soup.find_all("td",{"class":"titleColumn"})

In [8]:
len(movie_container)

250

In [12]:
movie_link=[]
for i in movie_container:
    a = i.find("a")
    movie_link.append("https://www.imdb.com/" + a.get("href"))

In [13]:
df_movie = pd.DataFrame(movie_link)

In [15]:
df_movie.to_excel("Movie_links.xlsx")

In [16]:
def get_Moviereview_links(l):
    agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
    review_links = []
    for i in l:
        a = i.split('?') [0]
        response = requests.get(i, headers=agent)

        # Create a Beautiful Soup object from the response content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all the review containers on the page
        table = soup.find_all('a',attrs={"class":"ipc-link ipc-link--baseAlt ipc-link--inherit-color",},text=True)
        for j in table:
            if j.has_attr('href'):
                if 'review' in str(j['href']):
                    if "title" in j["href"]:
                        review_links.append(a + str(j['href'][17:]))
                    else:
                        review_links.append(a + str(j['href']))
    return review_links

In [17]:
review_links = get_Moviereview_links(movie_link)

In [18]:
len(review_links)

250

In [19]:
df_reviews = pd.DataFrame(review_links)

In [21]:
df_reviews.to_excel('review_links.xlsx')

In [29]:
from selenium import webdriver
from bs4 import BeautifulSoup
def get_reviews(movie_url):
    all_review_links = []
    title_list = []
    for url in movie_url:
        driver = webdriver.Chrome()
        driver.get(url)
        try:
            for i in range(5):
                iframe = driver.find_element('id',"load-more-trigger").click()
                driver.implicitly_wait(15)
        except:
            print("Pages Not found for" + url)
            pass
        soup = BeautifulSoup(driver.page_source,'html.parser')
        driver.close()
        table = soup.find_all('a',attrs={"class":"title",},text=True)
        for j in table:
            if j.has_attr('href'):
                if '/review/' in str(j['href']):
                    all_review_links.append("https://www.imdb.com/" + str(j['href']))
                    title_list.append(url[21:38])
    return all_review_links,title_list

In [27]:
df_review_links = pd.read_excel("review_links.xlsx",names = ['Index', 'Link'])
list(df_review_links['Link'])

['https://www.imdb.com//title/tt0111161/reviews/?ref_=tt_ql_2',
 'https://www.imdb.com//title/tt0068646/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com//title/tt0468569/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com//title/tt0071562/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com//title/tt0050083/reviews/?ref_=tt_ql_2',
 'https://www.imdb.com//title/tt0108052/reviews/?ref_=tt_ql_2',
 'https://www.imdb.com//title/tt0167260/reviews/?ref_=tt_ql_2',
 'https://www.imdb.com//title/tt0110912/reviews/?ref_=tt_ql_2',
 'https://www.imdb.com//title/tt0120737/reviews/?ref_=tt_ql_2',
 'https://www.imdb.com//title/tt0060196/reviews/?ref_=tt_ql_2',
 'https://www.imdb.com//title/tt0109830/reviews/?ref_=tt_ql_2',
 'https://www.imdb.com//title/tt0137523/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com//title/tt0167261/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com//title/tt1375666/reviews/?ref_=tt_ql_2',
 'https://www.imdb.com//title/tt0080684/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com//titl

In [30]:
final_review_links, title_list = get_reviews(list(df_review_links['Link']))

Pages Not found forhttps://www.imdb.com//title/tt10872600//reviews/?ref_=tt_ql_2
Pages Not found forhttps://www.imdb.com//title/tt0071411/reviews/?ref_=tt_ql_urv


In [31]:
len(final_review_links)

31018

In [33]:
len(set(final_review_links))

31018

In [34]:
len(title_list)

31018

In [35]:
final_review_links[0]

'https://www.imdb.com//review/rw2284594/?ref_=tt_urv'

In [36]:
title_list[0]

'/title/tt0111161/'

In [38]:
df_all_titles = pd.DataFrame(title_list)
df_all_titles.to_excel('titles.xlsx')

In [41]:
df_all_review_links = pd.DataFrame(final_review_links)

In [43]:
df_all_review_links.to_excel('final_review_links.xlsx')

In [13]:
df_all_review_links = pd.read_excel("final_review_links.xlsx",names = ['Index', 'Review'])

In [15]:
df_all_review_links['Review'].iloc[500:1000]

500    https://www.imdb.com//review/rw0060025/?ref_=t...
501    https://www.imdb.com//review/rw0060034/?ref_=t...
502    https://www.imdb.com//review/rw3666418/?ref_=t...
503    https://www.imdb.com//review/rw5448378/?ref_=t...
504    https://www.imdb.com//review/rw1557823/?ref_=t...
                             ...                        
995    https://www.imdb.com//review/rw1166753/?ref_=t...
996    https://www.imdb.com//review/rw6192662/?ref_=t...
997    https://www.imdb.com//review/rw2434076/?ref_=t...
998    https://www.imdb.com//review/rw4388233/?ref_=t...
999    https://www.imdb.com//review/rw1308663/?ref_=t...
Name: Review, Length: 500, dtype: object

In [322]:
import time as t

def text_from_review(l,k):
    spoiler_list = []
    not_spoiler_list = []
    spoiler_title_list = []
    non_spoiler_title_list = []
    for i in range(len(l)):
        print("Url Count",i)
        r = requests.get(l[i], headers=agent)    # Reading the HTML Page

        soup = BeautifulSoup(r.content, 'html.parser') # Loading the HTMl to a beautifulSoup object

        table = soup.find_all('div',attrs={"class":"text show-more__control"})

        for j in table:
            spoiler = j.find_previous('span', {'class': 'spoiler-warning'})
            if spoiler==None:
                not_spoiler_list.append(str(table[0]))
                non_spoiler_title_list.append(k[i])
            else:
                spoiler_list.append(str(table[0]))
                spoiler_title_list.append(k[i])
        t.sleep(1)
        print("Going for next URL")
    return spoiler_list,not_spoiler_list,spoiler_title_list,non_spoiler_title_list

In [42]:
# Only Spoilers Code

import time as t

def text_from_review_only_spoilers(l,k):
    spoiler_list = []
    spoiler_title_list = []
    for i in range(len(l)):
        print("Url Count",i)
        r = requests.get(l[i], headers=agent)    # Reading the HTML Page

        soup = BeautifulSoup(r.content, 'html.parser') # Loading the HTMl to a beautifulSoup object

        table = soup.find_all('div',attrs={"class":"text show-more__control"})

        for j in table:
            spoiler = j.find_previous('span', {'class': 'spoiler-warning'})
            if spoiler!=None:
                spoiler_list.append(str(table[0]))
                spoiler_title_list.append(k[i])      
        t.sleep(1)
        print("Going for next URL")
    return spoiler_list,spoiler_title_list

In [43]:
# Only Spoilers code

count = 21500
spoilers_list = []
spoiler_title_list = []
for i in range(20):
    print("Checkpoint" + str(i))
    sl , stl = text_from_review_only_spoilers(final_review_links[count:count+500],title_list[count:count+500])
    spoilers_list.append(sl)
    spoiler_title_list.append(stl)
    count = count+500
    t.sleep(30)

Checkpoint0
Url Count 0
Going for next URL
Url Count 1
Going for next URL
Url Count 2
Going for next URL
Url Count 3
Going for next URL
Url Count 4
Going for next URL
Url Count 5
Going for next URL
Url Count 6
Going for next URL
Url Count 7
Going for next URL
Url Count 8
Going for next URL
Url Count 9
Going for next URL
Url Count 10
Going for next URL
Url Count 11
Going for next URL
Url Count 12
Going for next URL
Url Count 13
Going for next URL
Url Count 14
Going for next URL
Url Count 15
Going for next URL
Url Count 16
Going for next URL
Url Count 17
Going for next URL
Url Count 18
Going for next URL
Url Count 19
Going for next URL
Url Count 20
Going for next URL
Url Count 21
Going for next URL
Url Count 22
Going for next URL
Url Count 23
Going for next URL
Url Count 24
Going for next URL
Url Count 25
Going for next URL
Url Count 26
Going for next URL
Url Count 27
Going for next URL
Url Count 28
Going for next URL
Url Count 29
Going for next URL
Url Count 30
Going for next URL
Url Co

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [323]:
# Code for spoilers and Non Spoilers continues for remaining datapoints

count = 15500
spoilers_list = []
non_spoiler_list = []
spoiler_title_list = []
non_spoiler_title_list = []
for i in range(62):
    print("Checkpoint" + str(i))
    sl , nsl, stl,ntl  = text_from_review(final_review_links[count:count+500],title_list[count:count+500])
    spoilers_list.append(sl)
    non_spoiler_list.append(nsl)
    spoiler_title_list.append(stl)
    non_spoiler_title_list.append(ntl)
    count = count+500
    t.sleep(30)

Checkpoint0
Url Count 0
Going for next URL
Url Count 1
Going for next URL
Url Count 2
Going for next URL
Url Count 3
Going for next URL
Url Count 4
Going for next URL
Url Count 5
Going for next URL
Url Count 6
Going for next URL
Url Count 7
Going for next URL
Url Count 8
Going for next URL
Url Count 9
Going for next URL
Url Count 10
Going for next URL
Url Count 11
Going for next URL
Url Count 12
Going for next URL
Url Count 13
Going for next URL
Url Count 14
Going for next URL
Url Count 15
Going for next URL
Url Count 16
Going for next URL
Url Count 17
Going for next URL
Url Count 18
Going for next URL
Url Count 19
Going for next URL
Url Count 20
Going for next URL
Url Count 21
Going for next URL
Url Count 22
Going for next URL
Url Count 23
Going for next URL
Url Count 24
Going for next URL
Url Count 25
Going for next URL
Url Count 26
Going for next URL
Url Count 27
Going for next URL
Url Count 28
Going for next URL
Url Count 29
Going for next URL
Url Count 30
Going for next URL
Url Co

Going for next URL
Url Count 252
Going for next URL
Url Count 253
Going for next URL
Url Count 254
Going for next URL
Url Count 255
Going for next URL
Url Count 256
Going for next URL
Url Count 257
Going for next URL
Url Count 258
Going for next URL
Url Count 259
Going for next URL
Url Count 260
Going for next URL
Url Count 261
Going for next URL
Url Count 262
Going for next URL
Url Count 263
Going for next URL
Url Count 264
Going for next URL
Url Count 265
Going for next URL
Url Count 266
Going for next URL
Url Count 267
Going for next URL
Url Count 268
Going for next URL
Url Count 269
Going for next URL
Url Count 270
Going for next URL
Url Count 271
Going for next URL
Url Count 272
Going for next URL
Url Count 273
Going for next URL
Url Count 274
Going for next URL
Url Count 275
Going for next URL
Url Count 276
Going for next URL
Url Count 277
Going for next URL
Url Count 278
Going for next URL
Url Count 279
Going for next URL
Url Count 280
Going for next URL
Url Count 281
Going for 

Checkpoint1
Url Count 0
Going for next URL
Url Count 1
Going for next URL
Url Count 2
Going for next URL
Url Count 3
Going for next URL
Url Count 4
Going for next URL
Url Count 5
Going for next URL
Url Count 6
Going for next URL
Url Count 7
Going for next URL
Url Count 8
Going for next URL
Url Count 9
Going for next URL
Url Count 10
Going for next URL
Url Count 11
Going for next URL
Url Count 12
Going for next URL
Url Count 13
Going for next URL
Url Count 14
Going for next URL
Url Count 15
Going for next URL
Url Count 16
Going for next URL
Url Count 17
Going for next URL
Url Count 18
Going for next URL
Url Count 19
Going for next URL
Url Count 20
Going for next URL
Url Count 21
Going for next URL
Url Count 22
Going for next URL
Url Count 23
Going for next URL
Url Count 24
Going for next URL
Url Count 25
Going for next URL
Url Count 26
Going for next URL
Url Count 27
Going for next URL
Url Count 28
Going for next URL
Url Count 29
Going for next URL
Url Count 30
Going for next URL
Url Co

Going for next URL
Url Count 252
Going for next URL
Url Count 253
Going for next URL
Url Count 254
Going for next URL
Url Count 255
Going for next URL
Url Count 256
Going for next URL
Url Count 257
Going for next URL
Url Count 258
Going for next URL
Url Count 259
Going for next URL
Url Count 260
Going for next URL
Url Count 261
Going for next URL
Url Count 262
Going for next URL
Url Count 263
Going for next URL
Url Count 264
Going for next URL
Url Count 265
Going for next URL
Url Count 266
Going for next URL
Url Count 267
Going for next URL
Url Count 268
Going for next URL
Url Count 269
Going for next URL
Url Count 270
Going for next URL
Url Count 271
Going for next URL
Url Count 272
Going for next URL
Url Count 273
Going for next URL
Url Count 274
Going for next URL
Url Count 275
Going for next URL
Url Count 276
Going for next URL
Url Count 277
Going for next URL
Url Count 278
Going for next URL
Url Count 279
Going for next URL
Url Count 280
Going for next URL
Url Count 281
Going for 

Checkpoint2
Url Count 0
Going for next URL
Url Count 1
Going for next URL
Url Count 2
Going for next URL
Url Count 3
Going for next URL
Url Count 4
Going for next URL
Url Count 5
Going for next URL
Url Count 6
Going for next URL
Url Count 7
Going for next URL
Url Count 8
Going for next URL
Url Count 9
Going for next URL
Url Count 10
Going for next URL
Url Count 11
Going for next URL
Url Count 12
Going for next URL
Url Count 13
Going for next URL
Url Count 14
Going for next URL
Url Count 15
Going for next URL
Url Count 16
Going for next URL
Url Count 17
Going for next URL
Url Count 18
Going for next URL
Url Count 19
Going for next URL
Url Count 20
Going for next URL
Url Count 21
Going for next URL
Url Count 22
Going for next URL
Url Count 23
Going for next URL
Url Count 24
Going for next URL
Url Count 25
Going for next URL
Url Count 26
Going for next URL
Url Count 27
Going for next URL
Url Count 28
Going for next URL
Url Count 29
Going for next URL
Url Count 30
Going for next URL
Url Co

...

Going for next URL
Url Count 252
Going for next URL
Url Count 253
Going for next URL
Url Count 254
Going for next URL
Url Count 255
Going for next URL
Url Count 256
Going for next URL
Url Count 257
Going for next URL
Url Count 258
Going for next URL
Url Count 259
Going for next URL
Url Count 260
Going for next URL
Url Count 261
Going for next URL
Url Count 262
Going for next URL
Url Count 263
Going for next URL
Url Count 264
Going for next URL
Url Count 265
Going for next URL
Url Count 266
Going for next URL
Url Count 267
Going for next URL
Url Count 268
Going for next URL
Url Count 269
Going for next URL
Url Count 270
Going for next URL
Url Count 271
Going for next URL
Url Count 272
Going for next URL
Url Count 273
Going for next URL
Url Count 274
Going for next URL
Url Count 275
Going for next URL
Url Count 276
Going for next URL
Url Count 277
Going for next URL
Url Count 278
Going for next URL
Url Count 279
Going for next URL
Url Count 280
Going for next URL
Url Count 281
Going for 

Checkpoint16
Url Count 0
Going for next URL
Url Count 1
Going for next URL
Url Count 2
Going for next URL
Url Count 3
Going for next URL
Url Count 4
Going for next URL
Url Count 5
Going for next URL
Url Count 6
Going for next URL
Url Count 7
Going for next URL
Url Count 8
Going for next URL
Url Count 9
Going for next URL
Url Count 10
Going for next URL
Url Count 11
Going for next URL
Url Count 12
Going for next URL
Url Count 13
Going for next URL
Url Count 14
Going for next URL
Url Count 15
Going for next URL
Url Count 16
Going for next URL
Url Count 17
Going for next URL
Url Count 18
Going for next URL
Url Count 19
Going for next URL
Url Count 20
Going for next URL
Url Count 21
Going for next URL
Url Count 22
Going for next URL
Url Count 23
Going for next URL
Url Count 24
Going for next URL
Url Count 25
Going for next URL
Url Count 26
Going for next URL
Url Count 27
Going for next URL
Url Count 28
Going for next URL
Url Count 29
Going for next URL
Url Count 30
Going for next URL
Url C

Going for next URL
Url Count 252
Going for next URL
Url Count 253
Going for next URL
Url Count 254
Going for next URL
Url Count 255
Going for next URL
Url Count 256
Going for next URL
Url Count 257
Going for next URL
Url Count 258
Going for next URL
Url Count 259
Going for next URL
Url Count 260
Going for next URL
Url Count 261
Going for next URL
Url Count 262
Going for next URL
Url Count 263
Going for next URL
Url Count 264
Going for next URL
Url Count 265
Going for next URL
Url Count 266
Going for next URL
Url Count 267
Going for next URL
Url Count 268
Going for next URL
Url Count 269
Going for next URL
Url Count 270
Going for next URL
Url Count 271
Going for next URL
Url Count 272
Going for next URL
Url Count 273
Going for next URL
Url Count 274
Going for next URL
Url Count 275
Going for next URL
Url Count 276
Going for next URL
Url Count 277
Going for next URL
Url Count 278
Going for next URL
Url Count 279
Going for next URL
Url Count 280
Going for next URL
Url Count 281
Going for 

Checkpoint17
Url Count 0
Going for next URL
Url Count 1
Going for next URL
Url Count 2
Going for next URL
Url Count 3
Going for next URL
Url Count 4
Going for next URL
Url Count 5
Going for next URL
Url Count 6
Going for next URL
Url Count 7
Going for next URL
Url Count 8
Going for next URL
Url Count 9
Going for next URL
Url Count 10
Going for next URL
Url Count 11
Going for next URL
Url Count 12
Going for next URL
Url Count 13
Going for next URL
Url Count 14
Going for next URL
Url Count 15
Going for next URL
Url Count 16
Going for next URL
Url Count 17
Going for next URL
Url Count 18
Going for next URL
Url Count 19
Going for next URL
Url Count 20
Going for next URL
Url Count 21
Going for next URL
Url Count 22
Going for next URL
Url Count 23
Going for next URL
Url Count 24
Going for next URL
Url Count 25
Going for next URL
Url Count 26
Going for next URL
Url Count 27
Going for next URL
Url Count 28
Going for next URL
Url Count 29
Going for next URL
Url Count 30
Going for next URL
Url C

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [328]:
print(len(spoilers_list))
print(len(non_spoiler_list))
print(len(spoiler_title_list))
print(len(non_spoiler_title_list))

17
17
17
17


In [329]:
#spoiler dataframe

In [330]:
#Check the length of the lists - ** IMP

In [331]:
df_spoilers = pd.DataFrame(spoilers_list)

In [332]:

rows = list(zip(df_spoilers.T[0],spoiler_title_list[0])) + list(zip(df_spoilers.T[1],spoiler_title_list[1])) + list(zip(df_spoilers.T[2],spoiler_title_list[2])) + list(zip(df_spoilers.T[3],spoiler_title_list[3])) + list(zip(df_spoilers.T[4],spoiler_title_list[4])) + list(zip(df_spoilers.T[5],spoiler_title_list[5])) + list(zip(df_spoilers.T[6],spoiler_title_list[6])) + list(zip(df_spoilers.T[7],spoiler_title_list[7])) + list(zip(df_spoilers.T[8],spoiler_title_list[8])) + list(zip(df_spoilers.T[9],spoiler_title_list[9])) + list(zip(df_spoilers.T[10],spoiler_title_list[10])) + list(zip(df_spoilers.T[11],spoiler_title_list[11])) + list(zip(df_spoilers.T[12],spoiler_title_list[12])) + list(zip(df_spoilers.T[13],spoiler_title_list[13])) + list(zip(df_spoilers.T[14],spoiler_title_list[14])) + list(zip(df_spoilers.T[15],spoiler_title_list[15])) + list(zip(df_spoilers.T[16],spoiler_title_list[16]))

In [333]:
df = pd.DataFrame(rows, columns=['Review', 'Title'])

In [334]:
df.Review.nunique()

2000

In [335]:
df.to_excel("Spoiler_list7.xlsx")

In [336]:
#non spoiler dataframe

In [337]:
df_non_spoiler = pd.DataFrame(non_spoiler_list)

In [338]:
df_non_spoiler

,0,1,2,3,4,5,6,7,8,9,...,401,402,403,404,405,406,407,408,409,410
0,"<div class=""text show-more__control"">Mahavir S...","<div class=""text show-more__control"">With word...","<div class=""text show-more__control"">Watched #...","<div class=""text show-more__control"">What a st...","<div class=""text show-more__control"">Starting ...","<div class=""text show-more__control"">At first,...","<div class=""text show-more__control"">Dangal is...","<div class=""text show-more__control"">While the...","<div class=""text show-more__control"">While off...","<div class=""text show-more__control"">Dangal is...",...,"<div class=""text show-more__control"">There are...","<div class=""text show-more__control"">Two music...","<div class=""text show-more__control"">I'm sure ...","<div class=""text show-more__control"">I've only...","<div class=""text show-more__control"">No need t...","<div class=""text show-more__control"">Although ...","<div class=""text show-more__control"">Some Like...","<div class=""text show-more__control"">Although ...","<div class=""text show-more__control"">Everythin...","<div class=""text show-more__control"">It's a cl..."
1,"<div class=""text show-more__control"">This film...","<div class=""text show-more__control"">This is o...","<div class=""text show-more__control"">because i...","<div class=""text show-more__control"">This is t...","<div class=""text show-more__control"">A great c...","<div class=""text show-more__control"">Some Like...","<div class=""text show-more__control"">This film...","<div class=""text show-more__control"">I absolut...","<div class=""text show-more__control"">One of Bi...","<div class=""text show-more__control"">""Some Lik...",...,"<div class=""text show-more__control"">Perfect m...","<div class=""text show-more__control"">Such a gr...","<div class=""text show-more__control"">Most of t...","<div class=""text show-more__control"">When a gr...","<div class=""text show-more__control"">This movi...","<div class=""text show-more__control"">It gives ...","<div class=""text show-more__control"">GREEN BOO...","<div class=""text show-more__control"">Basically...",None,None
2,"<div class=""text show-more__control"">I underst...","<div class=""text show-more__control"">At the ce...","<div class=""text show-more__control"">I saw thi...","<div class=""text show-more__control"">It is dif...","<div class=""text show-more__control"">Based on ...","<div class=""text show-more__control"">This is n...","<div class=""text show-more__control"">Technical...","<div class=""text show-more__control"">How did t...","<div class=""text show-more__control"">This film...","<div class=""text show-more__control"">Great per...",...,None,None,None,None,None,None,None,None,None,None
3,"<div class=""text show-more__control"">This one ...","<div class=""text show-more__control"">The Truma...","<div class=""text show-more__control"">The Truma...","<div class=""text show-more__control"">The Truma...","<div class=""text show-more__control"">Truman Bu...","<div class=""text show-more__control"">One of th...","<div class=""text show-more__control"">I'm not g...","<div class=""text show-more__control"">Jim Carre...","<div class=""text show-more__control"">You're ri...","<div class=""text show-more__control"">Interesti...",...,None,None,None,None,None,None,None,None,None,None
4,"<div class=""text show-more__control"">SPOILER: ...","<div class=""text show-more__control"">A child p...","<div class=""text show-more__control"">I saw thi...","<div class=""text show-more__control"">The amazi...","<div class=""text show-more__control"">One word ...","<div class=""text show-more__control"">When peop...","<div class=""text show-more__control"">I find th...","<div class=""text show-more__control"">You reall...","<div class=""text show-more__control"">THE SIXTH...","<div class=""text show-more__control"">The boy H...",...,None,None,None,None,None,None,None,None,None,None
5,"<div class=""text show-more__control"">This is a...","<div class=""te

In [339]:
rows = list(zip(df_non_spoiler.T[0],non_spoiler_title_list[0])) + list(zip(df_non_spoiler.T[1],non_spoiler_title_list[1])) + list(zip(df_non_spoiler.T[2],non_spoiler_title_list[2])) + list(zip(df_non_spoiler.T[3],non_spoiler_title_list[3]))  + list(zip(df_non_spoiler.T[4],non_spoiler_title_list[4])) + list(zip(df_non_spoiler.T[5],non_spoiler_title_list[5])) + list(zip(df_non_spoiler.T[6],non_spoiler_title_list[6])) + list(zip(df_non_spoiler.T[7],non_spoiler_title_list[7])) + list(zip(df_non_spoiler.T[8],non_spoiler_title_list[8])) + list(zip(df_non_spoiler.T[9],non_spoiler_title_list[9])) + list(zip(df_non_spoiler.T[10],non_spoiler_title_list[10])) + list(zip(df_non_spoiler.T[11],non_spoiler_title_list[11])) + list(zip(df_non_spoiler.T[12],non_spoiler_title_list[12])) + list(zip(df_non_spoiler.T[13],non_spoiler_title_list[13])) + list(zip(df_non_spoiler.T[14],non_spoiler_title_list[14])) + list(zip(df_non_spoiler.T[15],non_spoiler_title_list[15]))+ list(zip(df_non_spoiler.T[16],non_spoiler_title_list[16]))

In [340]:
df = pd.DataFrame(rows, columns=['Review', 'Title'])

In [341]:
df.Review.nunique()

6497

In [342]:
df.head()

,Review,Title
0,"<div class=""text show-more__control"">Mahavir S...",/title/tt5074352/
1,"<div class=""text show-more__control"">With word...",/title/tt5074352/
2,"<div class=""text show-more__control"">Watched #...",/title/tt5074352/
3,"<div class=""text show-more__control"">What a st...",/title/tt5074352/
4,"<div class=""text show-more__control"">Starting ...",/title/tt5074352/


In [343]:
df.to_excel("Non_Spoiler_list7.xlsx")